Ce script récupère uniquement les variables "avant match" et rajoute des variables historiques pour affiner la régression logistique

In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("datatennis.csv")
df = df.sort_values(by=['tourney_date', 'match_id']).reset_index(drop=True)

In [3]:
# Initialisation 

# H2H
h2h_history = defaultdict(lambda: [0, 0]) 
# Winrate par tournoi                  
tourney_history = defaultdict(lambda: defaultdict(lambda: [0, 0])) 
# Forme (10 derniers matchs)
match_history = defaultdict(list)      
# Meilleur classement                
best_rank_history = {}                                      

p1_wh2h, p2_wh2h = [], []
p1_best_rank_col, p2_best_rank_col = [], []
form_score_p1, form_score_p2 = [], []
p1_tourney_winrate, p2_tourney_winrate = [], []

In [4]:
for _, row in df.iterrows():
    p1 = row['player1_name']
    p2 = row['player2_name']
    t_name = row['tourney_name']
    
    key = tuple(sorted([p1, p2]))
    stats_h2h = h2h_history[key]
    
    if p1 == key[0]:
        p1_wh2h.append(stats_h2h[0])
        p2_wh2h.append(stats_h2h[1])
    else:
        p1_wh2h.append(stats_h2h[1])
        p2_wh2h.append(stats_h2h[0])

    # Meilleur classement vu jusqu'ici
    r1_curr = row['player1_rank'] if not pd.isna(row['player1_rank']) else 9999
    r2_curr = row['player2_rank'] if not pd.isna(row['player2_rank']) else 9999
    
    # Le best rank est le min entre ce qu'on connaissait avant et le rank d'aujourd'hui
    b1 = min(best_rank_history.get(p1, 9999), r1_curr)
    b2 = min(best_rank_history.get(p2, 9999), r2_curr)
    
    p1_best_rank_col.append(b1 if b1 != 9999 else np.nan)
    p2_best_rank_col.append(b2 if b2 != 9999 else np.nan)

    # Forme
    score_p1 = sum(match_history[p1][-10:]) if len(match_history[p1]) >= 10 else 5
    score_p2 = sum(match_history[p2][-10:]) if len(match_history[p2]) >= 10 else 5
    form_score_p1.append(score_p1)
    form_score_p2.append(score_p2)

    # Winrate tournoi
    w1, l1 = tourney_history[p1][t_name]
    rate1 = w1 / (w1 + l1) if (w1 + l1) > 0 else 0.0
    p1_tourney_winrate.append(rate1)

    w2, l2 = tourney_history[p2][t_name]
    rate2 = w2 / (w2 + l2) if (w2 + l2) > 0 else 0.0
    p2_tourney_winrate.append(rate2)
    
    winner = p1 if row['player1_wins'] == 1 else p2
    
    # Update H2H
    if winner == key[0]:
        h2h_history[key][0] += 1
    else:
        h2h_history[key][1] += 1
    
    # Update meilleur rank
    best_rank_history[p1] = b1
    best_rank_history[p2] = b2

    # Update forme et tournoi
    if row['player1_wins'] == 1:
        match_history[p1].append(1)
        match_history[p2].append(0)
        tourney_history[p1][t_name][0] += 1
        tourney_history[p2][t_name][1] += 1
    else:
        match_history[p1].append(0)
        match_history[p2].append(1)
        tourney_history[p1][t_name][1] += 1
        tourney_history[p2][t_name][0] += 1

In [5]:
# Ajout des colonnes
df['player1_wh2h'] = p1_wh2h
df['player2_wh2h'] = p2_wh2h
df['player1_best_rank'] = p1_best_rank_col
df['player2_best_rank'] = p2_best_rank_col
df['player1_form_score'] = form_score_p1
df['player2_form_score'] = form_score_p2
df['player1_tourney_winrate'] = p1_tourney_winrate
df['player2_tourney_winrate'] = p2_tourney_winrate

# Liste des colonnes pré-match
colonnes_a_conserver = [
    'match_id', 'tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date', 'round',
    'player1_id', 'player1_name', 'player1_hand', 'player1_ht', 'player1_ioc', 'player1_age',
    'player2_id', 'player2_name', 'player2_hand', 'player2_ht', 'player2_ioc', 'player2_age',
    'player1_rank', 'player1_rank_points', 'player2_rank', 'player2_rank_points', 'year',
    'player1_wh2h', 'player2_wh2h',
    'player1_best_rank', 'player2_best_rank',
    'player1_form_score', 'player2_form_score',
    'player1_tourney_winrate', 'player2_tourney_winrate',
    'player1_wins'
]

df_pre_match = df[[col for col in colonnes_a_conserver if col in df.columns]]
df_pre_match.to_csv("datatennis_prematch.csv", index=False)